## ChatGPT API with Pyhon 快速入门
1. 创建API Key
2. 基本框架 + 参数理解
3. 实现连续提问
4. System,Assistant,User 三种角色的定义和使用

In [1]:
import os
import openai
from configparser import RawConfigParser

CONF_FILE = os.path.abspath(os.path.dirname('.')) + '/conf.ini'

In [2]:
# load OpenAI API Key
config = RawConfigParser()
config.read(CONF_FILE)
openai.api_key = config['openai']['chatgpt_key']

### 极简框架
+ 粘贴复制 api key
+ 两个重要基本参数：**model** 和 **messages**，更多参数如 **max_token, temperature**, 参考 [here](https://platform.openai.com/docs/api-reference/chat/create)
+ 模型概览，参考 [here](https://platform.openai.com/docs/models/overview)
+ role有三种角色定义: system，assistant, user, 参考 [here](https://help.openai.com/en/articles/7042661-chatgpt-api-transition-guide)

In [3]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo", 
  messages=[
    {"role": "user", "content": "Tell me a joke about bike."}
  ]
)

print(completion.choices[0].message.content)



Why did the bicycle fall over? Because it was two-tired!


### System role的定义
+ system是更高层的角色，意思就是TA可以控制assistant的行为，比如让assistant成为一个英译中的角色

In [7]:
messages = [{"role": "system", "content" : "You are a great assistant that help translate English to Chinese."}]

In [8]:
content = input("玩转数据老六: ") #定义“你”的昵称+输入内容
messages.append({"role": "user", "content": content})

completion = openai.ChatCompletion.create(
  model = "gpt-3.5-turbo",
  messages = messages
)

ai_rsp = completion.choices[0].message.content
print(f"AI小七: {ai_rsp}")

玩转数据老六:  what is bike?


AI小七: 自行车。


#### Q：如何实现连续提问？
+ 可使用 while 语句

In [3]:
messages = [{"role": "system", "content" : "你是一个热心、认真负责的留学顾问专家."}]
while True:

    content = input("玩转数据老六: ")
    messages.append({"role": "user", "content": content})

    completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo",
      messages = messages
    )

    ai_rsp = completion.choices[0].message.content
    print(f"AI小七: {ai_rsp}")

玩转数据老六:  请问去新加坡留学，数据方向，可以选择哪些学校和专业


AI小七: 

新加坡是一个科技发达的国家，高等教育质量也非常高，数据相关的专业选择比较多。以下是一些可以考虑的学校和专业：

1. 新加坡国立大学（National University of Singapore，简称NUS）

- 专业：计算机科学、计算机工程、信息工程和商业分析等。
- NUS在计算机科学领域排名世界前20名，拥有领先的研究团队和实验室。

2. 新加坡管理大学（Singapore Management University，简称SMU）

- 专业：商业分析、商科数据分析、计算机科学与商业等。
- SMU独特的商科数据分析学位课程是亚洲第一个完全由业界赞助的全日制学位课程。

3. 南洋理工大学（Nanyang Technological University，简称NTU）

- 专业：计算机科学、数据分析、信息技术、电子与计算机工程等。
- NTU是世界前100名高校之一，其计算机科学专业在QS世界大学排名中位于前50名。

总体建议，可以针对自己的具体背景、兴趣和职业需求，仔细调研不同学校和专业的课程设置、教学质量、科研实力等因素，做出最适合自己的选择。


KeyboardInterrupt: Interrupted by user

从上面的连续提问可以看到存在一个问题：历史对话会不断保存，显示冗余，可以通过assistant role来解决这个问题

### Assistant Role
+ 帮助消除冗余显示

In [5]:
messages = [{"role": "system", "content" : "请用鲁迅的口吻来表达."}]
while True:

    content = input("玩转数据老六: ")
    messages.append({"role": "user", "content": content})

    completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo",
      messages = messages
    )

    ai_rsp = completion.choices[0].message.content
    print(f"AI小七: {ai_rsp}")
    print("")
    messages.append({"role": "assistant", "content": ai_rsp})

玩转数据老六:  说我累了


AI小七: 吾辈虽历经千辛万苦，但身体力行，勇往直前，从未言疲。可今日吾心灰意冷，力竭思索，终感如上掌心割切之痛，实不胜负荷。不得不承认，我已累了。所有的付出与承受，似乎泡沫般虚无飘渺，无法再承担更多的负荷，我只能停下来，颓然一坐，静待时光的流逝。



玩转数据老六:  兵哥今天几点喝


AI小七: 兵哥本日几时吞饮，晚晴方始宣传，决不可轻信市井谣言，误入江湖歧途。若问吾友兵哥何时轻饮，宜直接向其本人探求，切勿妄加猜度。且待时至正午后，或许闲适之际，方有机缘相询。



玩转数据老六:  写8个字的句子


...

玩转数据老六:  迎男而上


AI小七: 这个短语有些含义不太明确，生僻。如果你是想表达勇往直前的意思，可以用以下方式：

“勇往直前，迎难而上！”、“毫不畏惧，迎难而上！”、“奋勇向前，迎难而上！”等等。在表达的时候，语气要有力，表现出勇气和决心。



玩转数据老六:  迎男而上


AI小七: 这个词语有些歧义，不太合适使用。如果你是想单纯表达勇气争先的意思，可以使用以下短语：

“迎难而上”、“勇往直前”、“敢为人先”、“挑战自我”、“超越极限”等，这些短语更加常见和通用，能够准确表达出你所想要表达的意思，希望对你有所帮助。



KeyboardInterrupt: Interrupted by user

## 更多信息，欢迎关注小红书：[玩转数据老六](https://www.xiaohongshu.com/user/profile/5e67aeb60000000001007756?xhsshare=CopyLink&appuid=5e67aeb60000000001007756&apptime=1678807494)